# DEBUG: for reloading AK model
## > not working on LUMI - to heavy for salloc (wtf?)

In [1]:
import xarray as xr
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import yaml
import pickle as pkl
import subprocess
from keras.regularizers import L1L2

# old import
# from src.data_preparation import mdl_dataset_prep as dataset_prep
# import src.feature_extraction.pca as feature_pca
# import src.utils.load_config as load_config

from src.data_preparation import load_data
from src.data_preparation.blocking_time_series_split import BlockingTimeSeriesSplit 
# from src.data_preparation import mdl_dataset_prep
from src.data_preparation import mdl_dataset

from src.modelling import mdl_input_dico  # input variables class
from src.modelling import mdl_params  # parameters class
from src.modelling import mdl_history

from src.modelling import model_autokeras
from src.modelling import super_model_dl

from src.visualization import mdl_non_recursive
from src.visualization import mdl_ypred_PCA
from src.visualization import mdl_introspect

from src.utils import reload_config
from src.utils import tardisml_utils

2023-02-17 12:51:24.266102: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/cray/pe/papi/6.0.0.15/lib64:/opt/cray/libfabric/1.15.0.0/lib64:/.singularity.d/libs
2023-02-17 12:51:24.266153: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# add argument to pass config file
import sys
# narg = len(sys.argv)  # number of arguments passed
# if narg > 1:
#     file_config = sys.argv[1]
# else:
file_config = '../config/config_to_jobs/config_AK_no_bias_0wk.yaml'

In [3]:
file_config

'../config/config_to_jobs/config_AK_no_bias_0wk.yaml'

In [6]:
rootdir = tardisml_utils.get_rootdir()
# file_config = '../config/data_proc_full.yaml'
conf = reload_config.Config(file_config, rootdir=rootdir, verbose=1)

Config file found: ../config/config_to_jobs/config_AK_no_bias_0wk.yaml
PCA results in: /scratch/project_465000269/edelleo1/Leo/results/pca_i100-550_j300-629
Config file updated 'pca_dir': ../config/config_to_jobs/config_AK_no_bias_0wk.yaml
Results in: /scratch/project_465000269/edelleo1/Leo/results/ak_230217-125214
Folder created

Subfolder created: /scratch/project_465000269/edelleo1/Leo/results/ak_230217-125214/ml/
Subfolder created: /scratch/project_465000269/edelleo1/Leo/results/ak_230217-125214/figures/
Config file updated 'results_dir': ../config/config_to_jobs/config_AK_no_bias_0wk.yaml
Config folders updated.
Config copied to: /scratch/project_465000269/edelleo1/Leo/results/ak_230217-125214
Config file found: /scratch/project_465000269/edelleo1/Leo/results/ak_230217-125214/config_AK_no_bias_0wk.yaml
Default config file is now the copied following one:
/scratch/project_465000269/edelleo1/Leo/results/ak_230217-125214/config_AK_no_bias_0wk.yaml


In [7]:
from importlib import reload

In [13]:
reload(mdl_dataset)
reload(load_data)

<module 'src.data_preparation.load_data' from '/users/edelleo1/tardis/tardis-ml/src/data_preparation/load_data.py'>

In [8]:
conf.setup

'no_bias'

In [9]:
conf.setup = 'no_bias_sia'

In [14]:
# ---------------------------------------------------
#                 Loading data
# ---------------------------------------------------
# ds = mdl_dataset.Dataset(conf, setup='no_bias')
ds = mdl_dataset.Dataset(conf, setup=conf.setup)

# H = [1]  # no impact whatsoever in autokeras for now !!!



Initialisation dataset...
History loaded from config file: /scratch/project_465000269/edelleo1/Leo/results/ak_230217-125214/config_AK_no_bias_0wk.yaml
Loading data...
Loading sia...
	Retrieve siconc
	Retrieve sisnthick
	Retrieve zos
	Retrieve vxsi
	Retrieve vysi
Loading forcing values...
Loading pca...
Apply land/ocean mask...
Retrieve PCs and EOFs
Adding sia
Size of the training set:    1958 days
Size of the validation set:   451 days
Size of the test set:         604 days
Config file updated 'ntrain': /scratch/project_465000269/edelleo1/Leo/results/ak_230217-125214/config_AK_no_bias_0wk.yaml
Config file updated 'nval': /scratch/project_465000269/edelleo1/Leo/results/ak_230217-125214/config_AK_no_bias_0wk.yaml
Config file updated 'ntest': /scratch/project_465000269/edelleo1/Leo/results/ak_230217-125214/config_AK_no_bias_0wk.yaml
Dataset Format for ak
No need for formatting


In [17]:
ds.inputs

['SITf t+0 PC0',
 'SITf t+0 PC1',
 'SITf t+0 PC2',
 'SITf t+0 PC3',
 'SITf t+0 PC4',
 'SITf t+0 PC5',
 'SITf t+0 PC6',
 'SITf t+0 PC7',
 '2T t+0 PC0',
 '2T t+0 PC1',
 '2T t+0 PC2',
 '2T t+0 PC3',
 'MSL t+0 PC0',
 'MSL t+0 PC1',
 'MSL t+0 PC2',
 'MSL t+0 PC3',
 '10V t+0 PC0',
 '10V t+0 PC1',
 '10V t+0 PC2',
 '10V t+0 PC3',
 '10U t+0 PC0',
 '10U t+0 PC1',
 '10U t+0 PC2',
 '10U t+0 PC3',
 'TP t+0 PC0',
 'TP t+0 PC1',
 'TP t+0 PC2',
 'TP t+0 PC3',
 'SSR t+0 PC0',
 'SSR t+0 PC1',
 'SSR t+0 PC2',
 'SSR t+0 PC3',
 'STR t+0 PC0',
 'STR t+0 PC1',
 'STR t+0 PC2',
 'STR t+0 PC3',
 'siconc t+0 PC0',
 'siconc t+0 PC1',
 'siconc t+0 PC2',
 'siconc t+0 PC3',
 'sisnthick t+0 PC0',
 'sisnthick t+0 PC1',
 'sisnthick t+0 PC2',
 'sisnthick t+0 PC3',
 'zos t+0 PC0',
 'zos t+0 PC1',
 'zos t+0 PC2',
 'zos t+0 PC3',
 'vxsi t+0 PC0',
 'vxsi t+0 PC1',
 'vxsi t+0 PC2',
 'vxsi t+0 PC3',
 'vysi t+0 PC0',
 'vysi t+0 PC1',
 'vysi t+0 PC2',
 'vysi t+0 PC3',
 'SIA t+0 PC0',
 'SIA t+0 PC1',
 'SIA t+0 PC2',
 'SIA t+0 PC

In [7]:
ds.dataset.keys()

dict_keys(['X', 'y', 'chrono', 'ntrain', 'nval', 'ntest', 'chrono_train', 'chrono_val', 'chrono_test', 'Xtrain', 'Xval', 'Xtest', 'ytrain', 'yval', 'ytest'])

In [8]:
ds.dataset['ytrain'] = ds.dataset['ytrain'][:100]
ds.dataset['Xtrain'] = ds.dataset['Xtrain'][:100]
ds.dataset['yval'] = ds.dataset['yval'][:100]
ds.dataset['Xval'] = ds.dataset['Xval'][:100]
ds.dataset['ytest'] = ds.dataset['ytest'][:100]
ds.dataset['Xtest'] = ds.dataset['Xtest'][:100]
ds.dataset['y'] = ds.dataset['y'][:300]
ds.dataset['X'] = ds.dataset['X'][:300]

In [9]:
n_components = ds.config.n_comp['tp']
# max_trials = ds.config.max_trials  # 100  # 100
epochs = ds.config.epochs  #100  # 100

mb = model_autokeras.ModelAK(ds=ds, rootdir=conf.rootdir, ml_dir=conf.ml_dir, fig_dir=conf.fig_dir)
mb.compile_models(npca=1)  # n_components)

2023-02-15 12:20:26.061756: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/cray/pe/papi/6.0.0.15/lib64:/opt/cray/libfabric/1.15.0.0/lib64:/.singularity.d/libs
2023-02-15 12:20:26.061791: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-15 12:20:26.061817: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (uan01): /proc/driver/nvidia/version does not exist
2023-02-15 12:20:26.062098: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
mb.explore_multiple(ds.dataset, epochs=epochs)  # dsk, batch_size=4)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
True              |?                 |structured_data_block_1/normalize
2                 |?                 |structured_data_block_1/dense_block_1/num_layers
False             |?                 |structured_data_block_1/dense_block_1/use_batchnorm
0                 |?                 |structured_data_block_1/dense_block_1/dropout
32                |?                 |structured_data_block_1/dense_block_1/units_0
32                |?                 |structured_data_block_1/dense_block_1/units_1
0                 |?                 |regression_head_1/dropout
adam              |?                 |optimizer
0.001             |?                 |learning_rate



In [1]:
## Once exploration done:
# access architecture and be able to recreate model without AutoKeras

-----------

In [ ]:
mb.predict_multiple(ds.dataset)  # dsk
mb.print_summary()

In [ ]:
# if mb.batch_size is None:
# mb.batch_size = 0
mb.suffix= ''
mb.print_histories()

mb.save_prediction()

# mb.save_model()
mb.save_model_weights()


ofile = f'{mb.type}_ypred.png'
mdl_ypred_PCA.draw(mb, odir=mb.rootdir+mb.fig_dir, savefig=True, showfig=False, ofile=ofile)